# Exercices

## **Préliminaires**: Clone de votre repo et imports

In [2]:
! git clone https://github.com/Gpoupart/exam_2025.git
! cp exam_2025/utils/utils_exercices.py .

import copy
import numpy as np
import torch

Cloning into 'exam_2025'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 59 (delta 21), reused 20 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (59/59), 1.41 MiB | 4.65 MiB/s, done.
Resolving deltas: 100% (21/21), done.


**Clef personnelle pour la partie théorique**

Dans la cellule suivante, choisir un entier entre 100 et 1000 (il doit être personnel). Cet entier servira de graine au générateur de nombres aléatoire a conserver pour tous les exercices.



In [3]:
mySeed = 205

\

---

\

\

**Exercice 1** *Une relation linéaire*

La fonction *generate_dataset* fournit deux jeux de données (entraînement et test). Pour chaque jeu de données, la clef 'inputs' donne accès à un tableau numpy (numpy array) de prédicteurs empilés horizontalement : chaque ligne $i$ contient trois prédicteurs $x_i$, $y_i$ et $z_i$. La clef 'targets' renvoie le vecteur des cibles $t_i$. \

Les cibles sont liées aux prédicteurs par le modèle:
$$ t = \theta_0 + \theta_1 x + \theta_2 y + \theta_3 z + \epsilon$$ où $\epsilon \sim \mathcal{N}(0,\eta)$


In [4]:
from utils_exercices import generate_dataset, Dataset1
train_set, test_set = generate_dataset(mySeed)

**Q1** Par quelle méthode simple peut-on estimer les coefficients $\theta_k$ ? La mettre en oeuvre avec la librairie python de votre choix.

la méthode la plus simple est la régression linéaire ordinaire ou méthode des moindres carrés. Cette méthode cherche à minimiser la somme des carrés des erreurs entre les valeurs prédites et les valeurs réelles des cibles t.

In [5]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

# Extraction des prédicteurs et cibles
X_train = train_set['inputs']  # Prédicteurs d'entraînement
y_train = train_set['targets']  # Cibles d'entraînement
X_test = test_set['inputs']  # Prédicteurs de test
y_test = test_set['targets']  # Cibles de test

# Création et entraînement du modèle
model = LinearRegression()
model.fit(X_train, y_train)

# Récupération des coefficients estimés
theta_0 = model.intercept_
theta_1, theta_2, theta_3 = model.coef_

# Affichage des coefficients
print(f"Estimation des coefficients :")
print(f"theta_0 = {theta_0:.4f}")
print(f"theta_1 = {theta_1:.4f}")
print(f"theta_2 = {theta_2:.4f}")
print(f"theta_3 = {theta_3:.4f}")

# Évaluation sur le jeu de test
y_pred = model.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred)
print(f"Erreur quadratique moyenne sur le jeu de test : {mse_test:.4f}")


Estimation des coefficients :
theta_0 = 10.2979
theta_1 = 2.1091
theta_2 = 2.0039
theta_3 = 4.1245
Erreur quadratique moyenne sur le jeu de test : 3.8136


**Q2** Dans les cellules suivantes, on se propose d'estimer les $\theta_k$ grâce à un réseau de neurones entraîné par SGD. Quelle architecture s'y prête ? Justifier en termes d'expressivité et de performances en généralisation puis la coder dans la cellule suivante.

Une seule couche linéaire (dense) sans fonction d'activation non linéaire, car la relation est linéaire.
Cette couche aura une entrée de dimension 3 (pour les prédicteurs x, y, et z) et une sortie de dimension 1 (pour la cible t).

Justification :

Cette architecture a la même capacité d’expressivité que la régression linéaire ordinaire.
Elle est suffisante pour capturer la structure des données et éviter un surajustement, tout en étant rapide à entraîner avec l’algorithme SGD (descente de gradient stochastique).

In [7]:
import torch.nn as nn

# Dataset et dataloader :
dataset = Dataset1(train_set['inputs'], train_set['targets'])
dataloader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=True)

class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.linear = nn.Linear(3, 1)  # Couche linéaire avec 3 entrées (x, y, z) et 1 sortie (t)

    def forward(self, x):
        return self.linear(x)  # Pas de fonction d'activation pour rester linéaire

**Q3** Entraîner cette architecture à la tâche de régression définie par les entrées et sorties du jeu d'entraînement (compléter la cellule ci-dessous).

In [8]:
# Initialize model, loss, and optimizer
mySimpleNet = SimpleNet()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(mySimpleNet.parameters(), lr=0.01)

# Training loop
num_epochs = 500
for epoch in range(num_epochs):
    for batch_inputs, batch_targets in dataloader:

        optimizer.zero_grad()
        outputs = mySimpleNet(batch_inputs)
        loss = criterion(outputs.squeeze(), batch_targets)
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 50 == 0:
        print(f"Époque [{epoch + 1}/{num_epochs}], Perte: {loss.item():.4f}")

Époque [50/500], Perte: 4.1781
Époque [100/500], Perte: 5.1251
Époque [150/500], Perte: 3.9170
Époque [200/500], Perte: 4.4401
Époque [250/500], Perte: 4.6016
Époque [300/500], Perte: 4.1645
Époque [350/500], Perte: 3.8194
Époque [400/500], Perte: 5.2983
Époque [450/500], Perte: 3.3848
Époque [500/500], Perte: 4.0985


**Q4** Où sont alors stockées les estimations des  $\theta_k$ ? Les extraire du réseau *mySimpleNet* dans la cellule suivante.

On accède aux paramètres de la couche linéaire :

Le biais représente θ0.

Les poids représentent θ1, θ2 et θ3.

In [9]:
theta_0 = mySimpleNet.linear.bias.item()
theta_1, theta_2, theta_3 = mySimpleNet.linear.weight[0].tolist()

print(f"Estimation des coefficients par le réseau de neurones :")
print(f"theta_0 = {theta_0:.4f}")
print(f"theta_1 = {theta_1:.4f}")
print(f"theta_2 = {theta_2:.4f}")
print(f"theta_3 = {theta_3:.4f}")


Estimation des coefficients par le réseau de neurones :
theta_0 = 10.2986
theta_1 = 2.1087
theta_2 = 2.0038
theta_3 = 4.1247


**Q5** Tester ces estimations sur le jeu de test et comparer avec celles de la question 1. Commentez.

In [13]:
from sklearn.metrics import mean_squared_error


test_inputs = torch.tensor(test_set['inputs'], dtype=torch.float32)
test_targets = torch.tensor(test_set['targets'], dtype=torch.float32)
test_predictions = mySimpleNet(test_inputs).squeeze().detach().numpy()


mse_nn = mean_squared_error(test_targets, test_predictions)
print(f"Erreur quadratique moyenne du réseau de neurones sur le jeu de test : {mse_nn:.4f}")

Erreur quadratique moyenne du réseau de neurones sur le jeu de test : 3.8137


On obtient quasiment le même résultat ce qui était attendu car les méthodes modélisent la même relation linéaire.

\

---

\

**Exercice 2** *Champ réceptif et prédiction causale*

Le réseau défini dans la cellule suivante est utilisé pour faire le lien entre les valeurs $(x_{t' \leq t})$ d'une série temporelle d'entrée et la valeur présente $y_t$ d'une série temporelle cible.

In [14]:
import torch.nn as nn
import torch.nn.functional as F
from utils_exercices import Outconv, Up_causal, Down_causal

class Double_conv_causal(nn.Module):
    '''(conv => BN => ReLU) * 2, with causal convolutions that preserve input size'''
    def __init__(self, in_ch, out_ch, kernel_size=3, dilation=1):
        super(Double_conv_causal, self).__init__()
        self.kernel_size = kernel_size
        self.dilation = dilation
        self.conv1 = nn.Conv1d(in_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn1 = nn.BatchNorm1d(out_ch)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn2 = nn.BatchNorm1d(out_ch)

    def forward(self, x):
        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        return x


class causalFCN(nn.Module):
    def __init__(self, dilation=1):
        super(causalFCN, self).__init__()
        size = 64
        n_channels = 1
        n_classes = 1
        self.inc = Double_conv_causal(n_channels, size)
        self.down1 = Down_causal(size, 2*size)
        self.down2 = Down_causal(2*size, 4*size)
        self.down3 = Down_causal(4*size, 8*size, pooling_kernel_size=5, pooling_stride=5)
        self.down4 = Down_causal(8*size, 4*size, pooling=False, dilation=2)
        self.up2 = Up_causal(4*size, 2*size, kernel_size=5, stride=5)
        self.up3 = Up_causal(2*size, size)
        self.up4 = Up_causal(size, size)
        self.outc = Outconv(size, n_classes)
        self.n_classes = n_classes

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up2(x5, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        return x

# Exemple d'utilisation
model = causalFCN()
# Série temporelle d'entrée (x_t):
input_tensor1 = torch.rand(1, 1, 10000)
# Série temporelle en sortie f(x_t):
output = model(input_tensor1)
print(output.shape)

torch.Size([1, 1, 10000])


**Q1** De quel type de réseau de neurones s'agit-il ? Combien de paramètres la couche self.Down1 compte-t-elle (à faire à la main) ?
Combien de paramètres le réseau entier compte-t-il (avec un peu de code) ?

Il s'agit d'un réseau de neurones convolutifs causaux.


Nb de paramètres dans self.Down1: (calcul "à la main")

conv1 : (canaux d’entree × canaux de sortie×taille du noyau)=64×128×3=24,576

BN1=2×128=256

conv2 : 128×128×3=49,152

BN2 : 2×128=256

Soit : 24,576(conv1)+256(BN1)+49,152(conv2)+256(BN2)=74,240


In [17]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

total_params = count_parameters(model)
print(f"Le réseau entier contient {total_params:,} paramètres.")


Le réseau entier contient 2,872,641 paramètres.


**Q2** Par quels mécanismes la taille du vecteur d'entrée est-elle réduite ? Comment est-elle restituée dans la deuxième partie du réseau ?

Réduction de la taille :

Max-pooling ou Average-pooling dans les couches de downsampling.

Convolutions avec des strides supérieurs à 1 dans certaines couches.


Restitution de la taille :

Up-sampling dans les couches Up_causal.
Fusion des caractéristiques via des connexions de saut entre les couches de downsampling et de up-sampling.

**Q3** Par quels mécanismes le champ réceptif est-il augmenté ? Préciser par un calcul la taille du champ réceptif en sortie de *self.inc*.

Mécanismes pour augmenter le champ réceptif :

1. Convolutions avec noyaux plus grands : Les noyaux de convolution étendent la zone d'entrée influençant chaque neurone de la sortie.

2. Dilatation : La dilatation augmente le champ réceptif en espaçant les éléments du noyau, sans augmenter la taille de la sortie.

3. Empilement de couches : Chaque couche supplémentaire augmente le champ réceptif global.

4. Stride : Un stride > 1 réduit la taille de la sortie et augmente le champ réceptif.

Calcul du champ réceptif en sortie de `self.inc` :

- Première convolution : Taille du noyau = 3, donc le champ réceptif après `conv1` est 3.
- Deuxième convolution : Taille du noyau = 3, donc le champ réceptif après `conv2` est 3.
- On doit soustraire 1 car le dernier élément de la première fenêtre est déjà pris en compte dans la deuxième fenêtre.

Le champ réceptif total est :

3 + 3 - 1 = 5

**Q4** Par un bout de code, déterminer empiriquement la taille du champ réceptif associé à la composante $y_{5000}$ du vecteur de sortie. (Indice: considérer les sorties associées à deux inputs qui ne diffèrent que par une composante...)

In [18]:
# Création d'un tensor d'entrée
input_tensor = torch.rand(1, 1, 10000)  # Exemple d'entrée de taille (1, 1, 10000)
output_tensor = model(input_tensor)  # Sortie du réseau

# Perturbation d'une seule composante de l'entrée (par exemple, la composante à l'index 5000)
input_tensor_perturbed = input_tensor.clone()
input_tensor_perturbed[0, 0, 5000] += 1e-5  # Perturbation très petite

# Passer l'entrée perturbée dans le modèle
output_tensor_perturbed = model(input_tensor_perturbed)

# Calcul de la différence dans la sortie pour la composante y5000
output_diff = output_tensor_perturbed[0, 0, 5000] - output_tensor[0, 0, 5000]

# Afficher la différence
print("Différence dans la sortie pour y5000:", output_diff.item())

# Mesurer l'effet de cette perturbation sur les autres valeurs de la sortie
# La taille du champ réceptif peut être déterminée par l'impact de cette perturbation


Différence dans la sortie pour y5000: -1.9744038581848145e-06


La petite différence dans la sortie signifie que la perturbation sur l'entrée à la position x5000 a légèrement affecté la sortie à la position y5000. Cette différence est très faible, ce qui indique que le champ réceptif de y5000 est relativement limité en termes de portée sur l'entrée.

**Q5** $y_{5000}$ dépend-elle des composantes $x_{t, \space t > 5000}$ ? Justifier de manière empirique puis préciser la partie du code de Double_conv_causal qui garantit cette propriété de "causalité" en justifiant.  



\

---

\

\

Exercice 3: "Ranknet loss"

Un [article récent](https://https://arxiv.org/abs/2403.14144) revient sur les progrès en matière de learning to rank. En voilà un extrait :


<img src="https://raw.githubusercontent.com/nanopiero/exam_2025/refs/heads/main/utils/png_exercice3.PNG" alt="extrait d'un article" width="800">

**Q1** Qu'est-ce que les auteurs appellent "positive samples" et "negative samples" ? Donner un exemple.

**Q2** Dans l'expression de $\mathcal{L}_{RankNet}$, d'où proviennent les $z_i$ ? Que représentent-ils ?  

**Q3** Pourquoi cette expression conduit-elle à ce que, après apprentissage, "the estimated
value of positive samples is greater than that of negative samples
for each pair of positive/negative samples" ?

**Q4** Dans le cadre d'une approche par deep learning, quels termes utilise-t-on pour qualifier les réseaux de neurones exploités et la modalité suivant laquelle ils sont entraînés ?